In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install nvcc4jupyter

In [3]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpyemc_15p".


In [4]:
%%cuda
#include <iostream>
    int
    main()
{
    std::cout << "Welcome To GeeksforGeeks\n";
    return 0;
}

Welcome To GeeksforGeeks



In [6]:
%%cuda
#include <cstdio>
#include <iostream>

    using namespace std;

__global__ void maxi(int* a, int* b, int n)
{
    int block = 256 * blockIdx.x;
    int max = 0;

    for (int i = block; i < min(256 + block, n); i++) {

        if (max < a[i]) {
            max = a[i];
        }
    }
    b[blockIdx.x] = max;
}

int main()
{

    int n;
    n = 1000; // 3 >> 2;
    int a[n];

    for (int i = 0; i < n; i++) {
        a[i] = rand() % n;
        cout << a[i] << "\t";
    }

    cudaEvent_t start, end;
    int *ad, *bd;
    int size = n * sizeof(int);
    cudaMalloc(&ad, size);
    cudaMemcpy(ad, a, size, cudaMemcpyHostToDevice);
    int grids = ceil(n * 1.0f / 256.0f);
    cudaMalloc(&bd, grids * sizeof(int));

    dim3 grid(grids, 1);
    dim3 block(1, 1);

    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);

    while (n > 1) {
        maxi<<<grids, block>>>(ad, bd, n);
        n = ceil(n * 1.0f / 256.0f);
        cudaMemcpy(ad, bd, n * sizeof(int), cudaMemcpyDeviceToDevice);
    }

    cudaEventRecord(end);
    cudaEventSynchronize(end);

    float time = 0;
    cudaEventElapsedTime(&time, start, end);

    int ans[2];
    cudaMemcpy(ans, ad, 4, cudaMemcpyDeviceToHost);

    cout << "The maximum element is : " << ans[0] << endl;

    cout << "The time required : ";
    cout << time << endl;
}


383	886	777	915	793	335	386	492	649	421	362	27	690	59	763	926	540	426	172	736	211	368	567	429	782	530	862	123	67	135	929	802	22	58	69	167	393	456	11	42	229	373	421	919	784	537	198	324	315	370	413	526	91	980	956	873	862	170	996	281	305	925	84	327	336	505	846	729	313	857	124	895	582	545	814	367	434	364	43	750	87	808	276	178	788	584	403	651	754	399	932	60	676	368	739	12	226	586	94	539	795	570	434	378	467	601	97	902	317	492	652	756	301	280	286	441	865	689	444	619	440	729	31	117	97	771	481	675	709	927	567	856	497	353	586	965	306	683	219	624	528	871	732	829	503	19	270	368	708	715	340	149	796	723	618	245	846	451	921	555	379	488	764	228	841	350	193	500	34	764	124	914	987	856	743	491	227	365	859	936	432	551	437	228	275	407	474	121	858	395	29	237	235	793	818	428	143	11	928	529	776	404	443	763	613	538	606	840	904	818	128	688	369	917	917	996	324	743	470	183	490	499	772	725	644	590	505	139	954	786	669	82	542	464	197	507	355	804	348	611	622	828	299	343	746	568	340	422	311	810	605	801	661	730	878	305